# Traffic Surveillance

This notebook will detail an implementations for traffic surveillance

## Prerequisites

Let's import all the necessary dependencies to make t

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

This notebook integrates the **NVIDIA CUDA Toolkit** to accelerate processes defined in the OpenCV libraries. The CUDA Toolkit provides the development environment and libraries necessary to run GPU-accelerated applications, while **PyTorch** provides a high-level interface for GPU-accelerated machine learning tasks using CUDA.

Following these steps you will be able to run Pytorch on cuda GPU:

- See if you have an NVIDIA gpu that supports *CUDA*. You can check whether your supports CUDA on the [Nvidia CUDA GPUs](https://developer.nvidia.com/cuda-gpus) list
- The *NVIDIA drivers* allow your operating system to communicate with the GPU. Download the latest driver for your GPU from [NVIDIA GPU drivers](https://www.nvidia.com/en-us/drivers/)
- The *NVIDIA CUDA® Deep Neural Network* library (cuDNN) is a GPU-accelerated library of primitives for deep neural networks. Download link: [Nvidia cuDNN](https://developer.nvidia.com/cudnn)
- The *CUDA Toolkit* provides the development environment and libraries necessary to run GPU-accelerated applications. Download the [Nvidia CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit), and add CUDA binaries to your system's PATH.
- Install *PyTorch with CUDA* support by visiting the [PyTorch installation](https://pytorch.org/get-started/locally/) page and choosing the appropriate options.

After completing the above steps, verify CUDA is available by running the following in a Python 

In [1]:
import torch

# Set up device working
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("Device used:", device)

PyTorch Version: 2.4.1+cu121
CUDA Available: False
CUDA Device Count: 0
Device used: cpu


/home/davevarga/Projects/virtenvs/bwim-venv/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Implementation

For a real-time traffic surveillance system that detects vehicle positions, reads license plates, and locates windshields, a multi-stage approach with different models can work best.

### Vehicle detection

At first, we need to detect the vehicles on our image. We not only need cars, but also trucks,  buses, and other large and heavy vehicles, as these yield the highest impact in bridge life.

### Dataset

The *dataset* choise is highly relevant, therefore we chose the **UA-DETRAC** dataset, which was specifically designed for vehicle detection and tracking in traffic surveillance systems. `Containing 140k images, with over 825k labeled vehicles from 10 different locations, it includes multiple vehicle types (car, bus, truck, van).

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download('dtrnngc/ua-detrac-dataset')

When training a model for traffic surveillance, dataset selection and preparation are crucial for achieving accurate and robust results. Here are key considerations regarding datasets:

- In case of BWIM surveillance systems, detection of trucks, and buses are considerably more important, as they account highly in *hid romlasi rata*. Therefore, we must ensure that the dataset *includes heavy vehicles* like trucks, buses, etc. 
- **Environmental conditions** play an important role in achieving robust models. Consequently different locations, camera angles, environmental conditions and times of day should be considered.
- **Data Quality:** High resolution images, not only allows the model to learn small details, but is crucial for following procedures, such as license plate recognition, while accurate labeling is important to ensure good performance. 
- **Data Balancing:** To optimize regression, every batch should sample proportionally across classes, environments and scenarios. In case of underrepresented classes over-sampling is suggested.
- **Domain Specific Data:** Although a balanced distribution is important, the dataset should mimic the real-world traffic surveillance system's use cases.
- **Data augmentation:** robustness can be improved using image transformations like noise injection, color and lightning manipulation, to improve performance under different scenarios, although might hurt training proficiency.
- **Normalization:** Normalizing the images (e.g., resizing, contrast normalization) ensures that they are fed into the model in a consistent format, therefore the model does not have to learn unnecessary features.

There is also a legal and ethical consideration, regarding data anonymity, license plate privacy, as well as dataset licensing. This responsibility falls on the user of the dataset.

In [6]:
import os
from ultralytics import YOLO

# Specify paths
dataset_yaml = './datasets/uc-detrac/uc-detrac.yaml'  # Path to dataset.yaml file
model_save_dir = './models'  # Directory where the model will be saved

# Create the directory if it doesn't exist
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

# Load the YOLOv8n (small) pre-trained model
model = YOLO('yolov8s.pt')  

# Train the model using the dataset
results = model.train(
    data=dataset_yaml,  # Path to the dataset YAML file
    epochs=10,          # Number of epochs (adjust based on your needs)
    batch=64,           # Batch size (adjust based on your hardware)
    imgsz=640,          # Image size (YOLOv8 typically uses 640x640)
    name='yolov8_traffic_surveillance',  # Name of the run
    device=device,
    fraction=0.1        # Allows for training on a subset of the full dataset
)

# Save the final model in ./models directory
model_path = os.path.join(model_save_dir, 'yolov8_car_detection.pt')
print(f"Model saved at {model_path}")


New https://pypi.org/project/ultralytics/8.3.12 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=./datasets/uc-detrac/uc-detrac.yaml, epochs=10, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolov8_traffic_surveillance, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fa

RuntimeError: Dataset 'datasets/uc-detrac/uc-detrac.yaml' error ❌ './datasets/uc-detrac/uc-detrac.yaml' does not exist